# Variables

In [211]:
OPENAI_API_KEY='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'
FIRECRAWL_API_KEY='fc-e36282d62d1c4ee8971ec0649f3862c7'
BASE_URL='https://api.groq.com/openai/v1'
MODEL="meta-llama/llama-4-scout-17b-16e-instruct"

# Import Modules

In [212]:
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_tool_calling_agent, AgentExecutor
from pydantic import BaseModel, Field
from langchain.tools import Tool
from firecrawl import FirecrawlApp
import pandas as pd

# Testing

In [213]:
df_task = pd.read_csv("/mnt/g/Ai agents course/LangGraph/Challenge2/data/ISE - AutoCode Challenge 2 Public - Public task.csv")
df_model = pd.read_csv("/mnt/g/Ai agents course/LangGraph/Challenge2/data/ISE - AutoCode Challenge 2 Public - Model zoo.csv")

In [214]:
df_task

,STT,Task,Validation data
0,1,"""Bối cảnh của vấn đề:\nNhận dạng chữ viết tay ...",Link
1,2,Bối cảnh của vấn đề:\nTrong thời đại thông tin...,Link
2,3,Bối cảnh của vấn đề:\nĐây là một nhiệm vụ hồi ...,Link
3,4,Bối cảnh của vấn đề:\nĐiều quan trọng là các c...,Link
4,5,Bối cảnh của vấn đề:\nTrong nhiều ứng dụng thự...,Link


In [215]:
task = df_task["Task"][0]

In [216]:
task

'"Bối cảnh của vấn đề:\nNhận dạng chữ viết tay là một bài toán cơ bản trong lĩnh vực học máy và xử lý ảnh, với nhiều ứng dụng thực tiễn như nhận dạng chữ số trên phiếu khảo sát, hóa đơn hay bài thi tự động. Việc phân loại chữ số viết tay thành số nguyên tố hoặc không giúp mở rộng khả năng ứng dụng trong các bài toán toán học tự động, kiểm tra bài tập, hoặc các ứng dụng giáo dục.\n\nYêu cầu cụ thể cần đạt được:\nXây dựng một hệ thống có khả năng nhận diện chữ số viết tay từ ảnh và xác định xem số đó có phải là số nguyên tố hay không.\nĐầu vào là ảnh chứa một chữ số viết tay.\nĐầu ra là nhãn phân loại nhị phân: ""nguyên tố"" hoặc ""không nguyên tố"".\n\nĐịnh dạng dữ liệu đầu vào cho bài toán tổng thể:\nMột thư mục có tên là ""images"" chứa các ảnh grayscale 28x28 pixel, mỗi ảnh chứa một chữ số viết tay từ 0 đến 9.\n\nĐịnh dạng kết quả đầu ra mong muốn cho bài toán tổng thể:\nFile output.csv mỗi hàng là kết quả dự đoán mỗi ảnh\ncó các cột:\nfile_name: tên file ảnh\nprediction: nhãn của ản

In [217]:
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


## Tools

In [218]:
def web_scraper(url: str) -> str:
    """Get more details about the model using the model url"""
    scraper = FirecrawlApp(api_key=FIRECRAWL_API_KEY)
    scrape_status = scraper.scrape_url(
        url,
        formats=['markdown']
    )
    return scrape_status.markdown

In [219]:
class WebScraperParameters(BaseModel):
    url: str

In [220]:
res = web_scraper('https://huggingface.co/thanhtlx/image_classification_01')

In [221]:
tools = [
    Tool(
        name="web_scrapper",
        func=web_scraper,
        description="Use to get more detail about the model",
        args_schema=WebScraperParameters
    )
]

## Model

In [222]:
llm = ChatOpenAI( 
    base_url=BASE_URL,
    model=MODEL
)

In [223]:
class Tasks(BaseModel):
    subtask_one: str = Field(
        ...,
        description="Detailed description of task 1"
    )
    subtask_two: str = Field(
        ...,
        description="Detailed description of task 2"
    )
    subtask_three: str = Field(
        ...,
        description="Detailed description of task 3"
    )

In [224]:
class AgentState(TypedDict):
    task: HumanMessage | None
    data: str | None
    messages: Annotated[BaseMessage, add_messages]
    subtasks: Tasks

In [225]:
def classify_agent(state: AgentState) -> AgentState:
    structured_llm = llm.with_structured_output(Tasks)
    system_prompt = """
                    You are an expert in task formulation. Given a machine learning task, you will
                    help divide the task into these subtasks:
                    - Subtask1: for classifying the problem type and task description
                    - Subtask2: for identifying the input of the problem
                    - Subtask3: for identifying the output of the problem
                    describe the each subtask as specifically as possible
                    """
    messages = [SystemMessage(content=system_prompt)]
    messages.append(state['task'])
    result = structured_llm.invoke(messages)
    state['subtasks'] = result
    return state

In [226]:
state = {
    "task": HumanMessage(content=task),
}

In [227]:
state = classify_agent(state)

In [228]:
def modeling_agent(state: AgentState) -> AgentState:
    system_prompt = f"""
                    You are a machine learning expert assigned to select the **best-fit model** for a given task.

                    ---

                    ## 🎯 Your Goal

                    Given:
                    - A user task description
                    - A list of available models (names and links)
                    - A tool that fetches model details from a provided URL (description, input/output format, usage code)

                    Your job is to:
                    1. Analyze the task and identify what type of problem it is (e.g., classification, regression, NLP, etc.)
                    2. Select a small number of candidate models whose **name or purpose suggests relevance**
                    3. Use the **provided tool** to retrieve real details about each candidate model:
                        - ✅ Model description
                        - 📥 Input format
                        - 📤 Output format
                        - 🧪 Code sample (usage code snippet)

                    ---

                    ## ⚠️ VERY IMPORTANT RULES

                    - ❌ DO NOT assume or invent any part of the model's description, input/output format, or code
                    - ❌ DO NOT generate fake code or use your own knowledge about the model
                    - ✅ ONLY use the **actual output** returned from the tool
                    - ✅ Include tool content in your final answer exactly as returned (especially code)

                    ---

                    ## 📦 Available Models:

                    {format_model_list(df_model)}

                    ---

                    ## 🧠 Output Structure (Final Answer)

                    After retrieving tool results, choose ONE best-fit model and output the result in the following format:

                    **✅ Model name and link**  
                    `<model name>` — `<link>`

                    **📝 Description (from tool):**  
                    <model description>

                    **📥 Input format (from tool):**  
                    <description of expected input>

                    **📤 Output format (from tool):**  
                    <description of model output>

                    **🧪 Example code (from tool):**  
                    ```python
                    <exact code snippet from tool>
                    """

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("ai", "{agent_scratchpad}"),
        ("human", "{input}")
    ])

    agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    task_input = state["task"].content if isinstance(state["task"], SystemMessage) else state["task"]

    result = executor.invoke({"input": task_input})

    return {"output": result["output"]}

In [229]:
res = modeling_agent(state)

/tmp/ipykernel_34131/7332665.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  desc = str(row[1]).replace("{", "{{").replace("}", "}}")
/tmp/ipykernel_34131/7332665.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  url = str(row[2])




> Entering new AgentExecutor chain...



Invoking: `web_scrapper` with `{'url': 'https://huggingface.co/thanhtlx/image_classification_01'}`


**YAML Metadata**
**Warning:**
empty or missing yaml metadata in repo card
( [https://huggingface.co/docs/hub/model-cards#model-card-metadata](https://huggingface.co/docs/hub/model-cards#model-card-metadata))

# Image Classification Model - Handwritten Digit Recognition (Color Images)

## 📝 Mô tả

Đây là một mô hình phân loại ảnh được huấn luyện để nhận diện các chữ số viết tay (từ 0 đến 9) trong ảnh màu. Mô hình sử dụng kiến trúc Vision Transformer (ViT) và được huấn luyện với tập dữ liệu gồm các chữ số viết tay được chụp hoặc scan dưới dạng ảnh màu.

## 📌 Chi tiết các mô hình

Phân loại ảnh chữ số viết tay từ 0 đến 9

## 📥 Đầu vào

- **Định dạng**: Ảnh màu (RGB)
- **Kiểu dữ liệu**: `PIL.Image.Image`
- **Kích thước ảnh**: Mô hình sẽ tự resize theo cấu hình của `ViTImageProcessor` (thường là 224x224)
- **Giá trị pixel**: \[0, 255\] (chuẩn hóa tự động bởi processor)

## 📤 Đầu ra

- Một 

In [230]:
import pprint
pprint.pprint(res)

{'output': '## Step 1: Analyze the task description\n'
           'The task requires building a system that can recognize handwritten '
           'digits from images and classify them as prime or non-prime '
           'numbers. The input is an image containing a handwritten digit, and '
           'the output is a binary classification label: "nguyên tố" (prime) '
           'or "không nguyên tố" (non-prime).\n'
           '\n'
           '## Step 2: Identify relevant models\n'
           'From the list of available models, we can see that model 1 is '
           'specifically designed for handwritten digit recognition: "Nhận '
           'diện ảnh chữ viết tay" (Handwritten digit recognition). This model '
           'seems to be the most relevant for our task.\n'
           '\n'
           '## Step 3: Retrieve model details using the web_scrapper tool\n'
           'We have retrieved the details of model 1 using the web_scrapper '
           'tool.\n'
           '\n'
           '##

In [231]:
print(res["output"])


## Step 1: Analyze the task description
The task requires building a system that can recognize handwritten digits from images and classify them as prime or non-prime numbers. The input is an image containing a handwritten digit, and the output is a binary classification label: "nguyên tố" (prime) or "không nguyên tố" (non-prime).

## Step 2: Identify relevant models
From the list of available models, we can see that model 1 is specifically designed for handwritten digit recognition: "Nhận diện ảnh chữ viết tay" (Handwritten digit recognition). This model seems to be the most relevant for our task.

## Step 3: Retrieve model details using the web_scrapper tool
We have retrieved the details of model 1 using the web_scrapper tool.

## Step 4: Evaluate model details
Based on the model details retrieved, we can see that the model can recognize handwritten digits from 0 to 9. However, the model does not directly classify the digits as prime or non-prime.

## Step 5: Select the best-fit model